# 01 - Merge Datasets

This notebook merges multiple h5ad files into a single AnnData object for integration.

## Inputs
- List of h5ad file paths
- Batch key name
- Optional metadata mapping

## Outputs
- `merged.h5ad` - Combined dataset
- `merge_summary.txt` - QC report

In [ ]:
import scanpy as sc
import anndata as ad
import pandas as pd
import numpy as np
from pathlib import Path
import yaml

# For nice progress bars
from tqdm.auto import tqdm

sc.settings.verbosity = 2

## Configuration

Edit the parameters below or load from a config file.

In [ ]:
# Option 1: Load from config file
# config_path = "../config/my_config.yaml"
# with open(config_path) as f:
#     config = yaml.safe_load(f)

# Option 2: Define parameters directly
config = {
    "input": {
        "files": [
            {"path": "/path/to/dataset1.h5ad", "name": "dataset1"},
            {"path": "/path/to/dataset2.h5ad", "name": "dataset2"},
        ],
        "batch_key": "dataset",  # Column name for batch after merge
    },
    "output": {
        "dir": "./results/",
    },
}

In [ ]:
# Extract config values
input_files = config["input"]["files"]
batch_key = config["input"]["batch_key"]
output_dir = Path(config["output"]["dir"])

# Create output directory
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Input files: {len(input_files)}")
print(f"Batch key: {batch_key}")
print(f"Output directory: {output_dir}")

## Load datasets

In [ ]:
adatas = {}

for file_info in tqdm(input_files, desc="Loading datasets"):
    path = file_info["path"]
    name = file_info["name"]
    
    print(f"\nLoading {name} from {path}")
    adata = sc.read_h5ad(path)
    
    # Add batch label
    adata.obs[batch_key] = name
    
    # Store
    adatas[name] = adata
    
    print(f"  Shape: {adata.shape}")
    print(f"  Obs columns: {list(adata.obs.columns)[:10]}...")

## Inspect datasets before merging

In [ ]:
# Summary table
summary = []
for name, adata in adatas.items():
    summary.append({
        "dataset": name,
        "n_cells": adata.n_obs,
        "n_genes": adata.n_vars,
        "has_raw": adata.raw is not None,
        "layers": list(adata.layers.keys()) if adata.layers else [],
    })

summary_df = pd.DataFrame(summary)
display(summary_df)

In [ ]:
# Check gene overlap
gene_sets = [set(adata.var_names) for adata in adatas.values()]
common_genes = set.intersection(*gene_sets)
all_genes = set.union(*gene_sets)

print(f"Common genes: {len(common_genes)}")
print(f"Total unique genes: {len(all_genes)}")
print(f"Overlap: {len(common_genes) / len(all_genes) * 100:.1f}%")

## Merge datasets

In [ ]:
# Use outer join to keep all genes, or inner to keep only common
join_type = "inner"  # or "outer"

print(f"Concatenating with join={join_type}...")
adata_merged = ad.concat(
    list(adatas.values()),
    join=join_type,
    label=batch_key,
    keys=list(adatas.keys()),
    index_unique="_",  # Add suffix to duplicate indices
)

print(f"\nMerged shape: {adata_merged.shape}")
print(f"Cells per batch:")
display(adata_merged.obs[batch_key].value_counts())

## Verify merge and clean up

In [ ]:
# Ensure batch_key is categorical
adata_merged.obs[batch_key] = adata_merged.obs[batch_key].astype("category")

# Remove any duplicate var columns that may have been created
# (keep first occurrence)
if adata_merged.var.columns.duplicated().any():
    adata_merged.var = adata_merged.var.loc[:, ~adata_merged.var.columns.duplicated()]

# Summary of obs columns
print("Merged obs columns:")
print(adata_merged.obs.columns.tolist())

In [ ]:
# Quick QC check
print("\nMissing values per column:")
missing = adata_merged.obs.isnull().sum()
missing = missing[missing > 0]
if len(missing) > 0:
    display(missing)
else:
    print("No missing values in obs.")

## Save merged dataset

In [ ]:
# Save full merged object
output_path = output_dir / "merged.h5ad"
print(f"Saving to {output_path}...")
adata_merged.write_h5ad(output_path)
print("Done!")

In [ ]:
# Save summary report
summary_path = output_dir / "merge_summary.txt"
with open(summary_path, "w") as f:
    f.write("=== Merge Summary ===\n\n")
    f.write(f"Total cells: {adata_merged.n_obs}\n")
    f.write(f"Total genes: {adata_merged.n_vars}\n")
    f.write(f"Batch key: {batch_key}\n\n")
    f.write("Cells per batch:\n")
    for batch, count in adata_merged.obs[batch_key].value_counts().items():
        f.write(f"  {batch}: {count}\n")
    f.write(f"\nOutput: {output_path}\n")

print(f"Summary saved to {summary_path}")

## Optional: Filter by cell type

Uncomment and modify to create a filtered version with specific cell types.

In [ ]:
# # Filter to specific cell types
# celltype_col = "celltype"  # Adjust to your column name
# keep_types = ["alpha", "beta", "delta"]  # Adjust to your cell types

# if celltype_col in adata_merged.obs.columns:
#     mask = adata_merged.obs[celltype_col].isin(keep_types)
#     adata_filt = adata_merged[mask].copy()
#     print(f"Filtered to {adata_filt.n_obs} cells")
#     
#     # Save filtered
#     filt_path = output_dir / "merged_celltype_filt.h5ad"
#     adata_filt.write_h5ad(filt_path)
#     print(f"Saved to {filt_path}")

## Next steps

The merged dataset is now ready for integration. Proceed to one of:

- `02_harmony_integration.ipynb` - Harmony (via R/rpy2)
- `03_seurat_integration.ipynb` - Seurat methods (CCA, RPCA, FastMNN)
- `04_python_integration.ipynb` - scVI and Scanorama (pure Python)